# *ferret* ❤️ *thermostat*

Hi there! This notebook show how **ferret** easily integrates **[thermostat](https://github.com/DFKI-NLP/thermostat)**, a large collection of pre-computed NLP model explanations.

Specifically, you will see how to:

- load a thermostat dataset;
- visualize the explanations of thermostat dataset of multiple explainers;
- **evaluate** thermostat explanations over the faithfulness metrics.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from ferret.datasets.datamanagers_thermostat import ThermostatDataset
from ferret import Benchmark
import numpy as np

/Users/eliana/miniconda3/envs/xai/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Thermostat explanations

thermostat is a collection of pre-computed explanations of NLP models. Explanations are pre-computed for a wide range of Hugging Face's datasets and transformers-based models.



ferret easily integrates thermostat.
A thermostat dataset is identified by the three basic information: dataset, model, and explainer.
For loading it, ferret adopts the same input configuration. 

For example, we can load the explanations of lime for the bert model on the ag news dataset by specifying the string "ag_news-bert-lime": `thermostat_data_exp = ThermostatDataset("ag_news-bert-lime")`

The power of **ferret** is its ability to **compare** explanations for **multiple explainers and evaluate** them.

ferret directly integrates explanations of multiple explainers, directly available in Thermostat! 🚀 

In the example, we load the explanations of the *lime*, *occlusion* and *layer integrated gradients* (lig) explainers for the bert prediction on the ag news dataset. 

In [3]:
thermostat_data = ThermostatDataset("ag_news-bert", name_explainers=["lime", "occlusion", "lig"])

Reusing dataset dataset (/Users/eliana/.cache/huggingface/datasets/dataset/ag_news-bert-lime/1.0.2/0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b)


Loading Thermostat configuration: ag_news-bert-lime


Reusing dataset dataset (/Users/eliana/.cache/huggingface/datasets/dataset/ag_news-bert-occlusion/1.0.2/0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b)


Loading Thermostat configuration: ag_news-bert-occlusion


Reusing dataset dataset (/Users/eliana/.cache/huggingface/datasets/dataset/ag_news-bert-lig/1.0.2/0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b)


Loading Thermostat configuration: ag_news-bert-lig


We access to the model and tokenizer adopted by the imported thermostat dataset. We then directly use our `Benchmark` interface class.

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(thermostat_data.model_name)
tokenizer = AutoTokenizer.from_pretrained(thermostat_data.tokenizer_name)


from ferret import Benchmark

bench = Benchmark(model, tokenizer)

## Explanations of a single instance and its evaluation

We can easily get the explanations for an instance and visualize it.

Note that we can get explanations by its index (`explanations = thermostat_data[i]["explanations"]`) or directly via the `get_explanations` function.

In [5]:
i = 0
# We normalize explanations to ease the comparison
explanations = thermostat_data.get_explanations(i, normalize_scores = True)

table = bench.show_table(explanations)
table

,fears,for,t,n,pension,after,talks,unions,representing,workers,at,turner,new,##all,say,they,are,',disappointed,'.1,after.1,talks.1,with,stricken,parent,firm,federal,mo,##gul,.
lime,0.02,-0.01,-0.04,-0.14,0.02,0.11,0.03,0.01,-0.01,0.00,-0.01,0.02,-0.02,0.00,-0.06,-0.04,-0.14,-0.01,-0.00,-0.07,0.01,0.01,0.08,-0.00,-0.00,0.05,0.01,0.00,0.01,-0.05
occlusion,-0.10,-0.04,0.04,0.14,0.10,0.05,-0.03,-0.02,-0.01,-0.01,0.00,0.00,0.00,-0.02,-0.05,-0.04,-0.01,-0.00,0.01,-0.03,-0.01,-0.01,0.01,0.01,-0.01,-0.02,-0.02,-0.04,0.02,0.03
lig,0.06,0.02,-0.07,0.03,0.13,0.03,0.03,0.07,0.01,0.02,0.00,0.01,0.00,-0.01,-0.01,-0.01,-0.04,-0.01,0.04,-0.02,-0.03,0.03,-0.00,0.02,0.06,0.09,0.03,-0.06,0.03,-0.02


Evaluate the pre-computed explanations with all the supported evaluators is straightforward. 
The `target` parameter matches the one used by Thermostat to generate the the explanations of the instance!

In [6]:
target = thermostat_data.get_target_explanations(i)
explanation_evaluations = bench.evaluate_explanations(explanations, target=target)

We can then visualize the result

In [7]:
bench.show_evaluation_table(explanation_evaluations)

,aopc_compr,aopc_suff,taucorr_loo
lime,0.19,0.15,-0.01
occlusion,0.13,-0.00,0.02
lig,0.24,-0.00,-0.06


## Evaluate explainers on a multiple instances

We can then evaluate explanations across multiple samples. The evaluation compute the average faithfulness scores of the explainers across the samples 

In [8]:
# Compute and average evaluation scores one of the supported dataset
samples = np.arange(10)
sample_evaluations =  bench.evaluate_samples(thermostat_data, samples)

and we visualize the result

In [9]:
bench.show_samples_evaluation_table(sample_evaluations)

,aopc_compr,aopc_suff,taucorr_loo
lime,0.208240,0.160388,0.031872
occlusion,0.282744,0.081829,0.231818
lig,0.407021,0.017823,0.056728
